In [45]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import scikeras
import os
import pickle

In [46]:
# method used for saving object as pickle
def save_object_as_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

In [47]:
# LOAD THE DATA 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

### Tensorboard

In [3]:
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(parameter, value):
    import time
    run_id = str(int(time.time())) + "_" + parameter + "_" + str(value)
    return os.path.join(root_logdir, run_id)

#### Callbacks

In [35]:
def get_callbacks_list(parameter, value):
    checkpoint_cb = keras.callbacks.ModelCheckpoint(parameter + "_" +  str(value)  + ".h5")
    run_logdir = get_run_logdir(parameter, value)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, monitor="loss")

    callbacks_list=[
        # checkpoint_cb,
        tensorboard_cb,
        early_stopping_cb]
    return callbacks_list

### Create build_model method

In [30]:
def build_model(n_hidden=1, n_neurons=25, learning_rate=10e-5, input_shape=[13], optimizer="sgd", momentum=0):
    model = keras.models.Sequential() 
    model.add(keras.layers.InputLayer(input_shape=input_shape)) 
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    if optimizer == "sgd":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True, momentum=momentum)
    elif optimizer == "momentum":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    elif optimizer == "adam":
        optimizer_prep = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=['mse', 'mae'], optimizer=optimizer_prep, metrics=['mse', 'mae'])
    return model

In [6]:
# model = build_model(input_shape=13)

In [7]:
# model.fit(
#     X_train,
#     y_train,
#     epochs=100,
#     verbose=1,
#     callbacks=callbacks_list
# )

### Learning rate test 

In [31]:
lr_test = [10**(-6),  10**(-5), 10**(-4)]
lr_values = []
lr_mse = []
lr_mae = []

In [36]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for lr_val in lr_test:
    callbacks_list = get_callbacks_list("lr", lr_val)
    model = build_model(input_shape=13, learning_rate=lr_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    lr_values.append(lr_val)
    print(lr_val)
    mse_val = np.mean(history.history['mse']) 
    lr_mse.append(mse_val)
    print(mse_val)
    mae_val = np.mean(history.history['mae']) 
    lr_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

1e-06
149.490888376705
7.869622339967822
-------------
1e-05
87.8173098986662
6.113666208484505
-------------
0.0001
1638718.4870922852
47.63900451660156
-------------


### Tensorboard analysis

In [10]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Hidden layers test 

In [11]:
hl_test = [0, 1, 2, 3]
hl_values = []
hl_mse = []
hl_mae = []


In [37]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for hl_val in hl_test:
    callbacks_list = get_callbacks_list("hl", hl_val)
    model = build_model(input_shape=13, n_hidden=hl_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    hl_values.append(lr_val)
    print(lr_val)
    mse_val = np.mean(history.history['mse']) 
    hl_mse.append(mse_val)
    print(mse_val)
    mae_val = np.mean(history.history['mae']) 
    hl_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

0.0001
nan
nan
-------------
0.0001
4279.35326751709
21.78880132675171
-------------
0.0001
857837160898519.0
1008718.78875
-------------
0.0001
nan
nan
-------------


### Tensorboard analysis

In [13]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### N_neurons test 

In [38]:
nn_test = [5, 25, 125]
nn_values = []
nn_mse = []
nn_mae = []


In [39]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for nn_val in nn_test:
    callbacks_list = get_callbacks_list("nn", nn_val)
    model = build_model(input_shape=13, n_neurons=nn_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    nn_values.append(nn_val)
    print(nn_val)
    mse_val = np.mean(history.history['mse']) 
    nn_mse.append(mse_val)
    print(mse_val)
    mae_val = np.mean(history.history['mae']) 
    nn_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

5
7306.537048950196
22.521386528015135
-------------
25
677.59498046875
20.260784854888914
-------------
125
1718045.812805786
65.11436664581299
-------------


### Tensorboard analysis

In [16]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Optimization algorithm test 

In [40]:
opt_test = ["sgd", "nesterov", "momentum", "adam"]
opt_values = []
opt_mse = []
opt_mae = []

In [41]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for opt_val in opt_test:
    callbacks_list = get_callbacks_list("opt", opt_val)
    model = build_model(input_shape=13, optimizer=opt_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    opt_values.append(opt_val)
    print(opt_val)
    mse_val = np.mean(history.history['mse']) 
    opt_mse.append(mse_val)
    print(mse_val)
    mae_val = np.mean(history.history['mae']) 
    opt_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

sgd
887376.3902449035
40.206524477005004
-------------
nesterov
677.59498046875
20.260784854888914
-------------
momentum
1638718.4870922852
47.63900451660156
-------------
adam
8497.9223828125
63.36647847175598
-------------


### Tensorboard analysis

In [19]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Momentum test 

In [42]:
mom_test = [0.1, 0.5, 0.9]
mom_values = []
mom_mse = []
mom_mae = []

In [43]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for mom_val in mom_test:
    callbacks_list = get_callbacks_list("mom", mom_val)
    model = build_model(input_shape=13, optimizer='momentum', momentum=mom_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    mom_values.append(mom_val)
    print(mom_val)
    mse_val = np.mean(history.history['mse']) 
    mom_mse.append(mse_val)
    print(mse_val)
    mae_val = np.mean(history.history['mae']) 
    mom_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

0.1
887354.2638946533
39.43483898162842
-------------
0.5
599.756106262207
18.06650921821594
-------------
0.9
1639428.7137620545
60.61562658309936
-------------


### Tensorboard analysis

In [44]:
# %load_ext tensorboard
# %tensorboard --logdir ./tb_logs --port=6006

## Save values in pickle

In [49]:
# learning rate
lr_data = []
for i in range(len(lr_test)):
    lr_data.append((lr_values[i], lr_mse[i], lr_mae[i]))

print(lr_data)
save_object_as_pickle(lr_data, "lr.pkl")

[(1e-06, 149.490888376705, 7.869622339967822), (1e-05, 87.8173098986662, 6.113666208484505), (0.0001, 1638718.4870922852, 47.63900451660156)]


In [50]:
# hidden layers 
hl_data = []
for i in range(len(hl_test)):
    hl_data.append((hl_values[i], hl_mse[i], hl_mae[i]))

print(hl_data)
save_object_as_pickle(hl_data, "hl.pkl")

[(0.0001, nan, nan), (0.0001, 4279.35326751709, 21.78880132675171), (0.0001, 857837160898519.0, 1008718.78875), (0.0001, nan, nan)]


In [51]:
# n_neurons
nn_data = []
for i in range(len(nn_test)):
    nn_data.append((nn_values[i], nn_mse[i], nn_mae[i]))

print(nn_data)
save_object_as_pickle(nn_data, "nn.pkl")

[(5, 7306.537048950196, 22.521386528015135), (25, 677.59498046875, 20.260784854888914), (125, 1718045.812805786, 65.11436664581299)]


In [56]:
# optimization
opt_data = []
for i in range(len(opt_test)):
    opt_data.append((opt_values[i], opt_mse[i], opt_mae[i]))

print(opt_data)
save_object_as_pickle(opt_data, "nn.pkl")

[('sgd', 887376.3902449035, 40.206524477005004), ('nesterov', 677.59498046875, 20.260784854888914), ('momentum', 1638718.4870922852, 47.63900451660156), ('adam', 8497.9223828125, 63.36647847175598)]


In [59]:
# momentum
mom_data = []
for i in range(len(mom_test)):
    mom_data.append((mom_values[i], mom_mse[i], mom_mae[i]))

print(mom_data)
save_object_as_pickle(mom_data, "mom.pkl")

[(0.1, 887354.2638946533, 39.43483898162842), (0.5, 599.756106262207, 18.06650921821594), (0.9, 1639428.7137620545, 60.61562658309936)]


### TASK 2.2

In [60]:
# TODO